In [1]:
import argparse
import tensorflow as tf
import glovedata
from glovedata import FEATURES
#import tensorglove_osc_server

/Users/Mike/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/Users/Mike/anaconda/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
batch_size = 1000
train_steps = 1000

In [3]:
def serving_input_receiver_fn():
    """Build the serving inputs."""
    inputs = {}
    for feat in FEATURES:
        inputs[feat] = tf.placeholder(shape=[None], dtype='float32')

    features = {
        key: tf.expand_dims(tensor, -1)
        for key, tensor in inputs.items()
    }

    return tf.estimator.export.ServingInputReceiver(features,
                                                    inputs)

In [4]:
# Fetch the data
(train_x, train_y), (test_x, test_y) = glovedata.load_data()

/Users/Mike/Documents/Code/side-projects/tensorglove/python/glovedata.py:53: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  data.convert_objects()
/Users/Mike/Documents/Code/side-projects/tensorglove/python/glovedata.py:58: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  train_y.convert_objects()
/Users/Mike/Documents/Code/side-projects/tensorglove/python/glovedata.py:60: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  test_y.convert_objects()


In [5]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in train_x.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

# Build 2 hidden layer DNN with 10, 10 units respectively.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 10 nodes each.
    hidden_units=[10, 10],
    # The model must choose between 4 classes.
    n_classes=4,
    model_dir="model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_log_step_count_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x102ec7c88>, '_model_dir': 'model', '_is_chief': True, '_tf_random_seed': None, '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000, '_save_summary_steps': 100, '_num_ps_replicas': 0, '_global_id_in_cluster': 0, '_num_worker_replicas': 1, '_train_distribute': None, '_session_config': None, '_evaluation_master': '', '_master': '', '_service': None, '_task_type': 'worker', '_task_id': 0}


In [ ]:
# Train the Model.
classifier.train(
    input_fn=lambda: glovedata.train_input_fn(train_x, train_y,
                                              batch_size),
    steps=train_steps)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into model/model.ckpt.
INFO:tensorflow:step = 1, loss = 1467.6824
INFO:tensorflow:global_step/sec: 11.2885
INFO:tensorflow:step = 101, loss = 391.88068 (8.859 sec)
INFO:tensorflow:global_step/sec: 11.9132
INFO:tensorflow:step = 201, loss = 202.16824 (8.394 sec)
INFO:tensorflow:global_step/sec: 11.2657
INFO:tensorflow:step = 301, loss = 140.69455 (8.877 sec)
INFO:tensorflow:global_step/sec: 12.2037
INFO:tensorflow:step = 401, loss = 106.72014 (8.194 sec)
INFO:tensorflow:global_step/sec: 12.6288
INFO:tensorflow:step = 501, loss = 106.3407 (7.918 sec)
INFO:tensorflow:global_step/sec: 12.1446
INFO:tensorflow:step = 601, loss = 73.63194 (8.234 sec)
INFO:tensorflow:global_step/sec: 13.0061
INFO:tensorflow:step = 701, l

In [ ]:
# Evaluate the model.
eval_result = classifier.evaluate(
    input_fn=lambda: glovedata.eval_input_fn(test_x, test_y,
                                             args.batch_size))


print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

In [ ]:
# perform a sample prediction

# Generate predictions from the model
expected = ['Fist']
feature_values = [0, 0, 0, 1, 0.2298394, 0.9228836, -0.1887825, -0.2445833, 0.03349165, -0.2397906, -0.1800468,
                  0.9533949, -0.07355202, 0.2745002, -0.2481479, 0.9261007, -4.470348E-08, 0.2213377, 1.490116E-08,
                  0.9751973, -2.235174E-08, -5.960464E-08, 4.470348E-08, 1, 0.09148686, -0.09309785, -0.6949129,
                  0.7071485, 7.105427E-15, 7.45058E-08, -0.7191889, 0.6948146, 1.490116E-08, -1.490116E-08,
                  -0.566272, 0.8242185, -2.235174E-08, -5.960464E-08, 4.470348E-08, 1, -3.725291E-08, 5.960464E-08,
                  -0.7009093, 0.7132504, 1.490116E-08, 3.352762E-08, -0.772563, 0.6349381, -7.450572E-09,
                  2.980232E-08, -0.6148145, 0.7886717, -2.235174E-08, -5.960464E-08, 4.470348E-08, 1, -0.06108833,
                  0.06216392, -0.6982422, 0.7105362, -3.352762E-08, 1.490116E-08, -0.7539417, 0.6569413,
                  5.960465E-08, 5.960463E-08, -0.5976215, 0.8017784, -2.235174E-08, -5.960464E-08, 4.470348E-08, 1,
                  -0.1217117, 0.1238547, -0.6902609, 0.7024146, -5.029142E-08, 3.725291E-08, -0.819152, 0.5735765,
                  5.215406E-08, 5.960463E-08, -0.6427875, 0.7660446]
# the data is expected to be a list of feature values (as it is configured for batching)
predict_x = dict(zip(FEATURES, [[x] for x in feature_values]))

predictions = classifier.predict(
    input_fn=lambda: glovedata.eval_input_fn(predict_x,
                                             labels=None,
                                             batch_size=args.batch_size))

template = '\nPrediction is "{}" ({:.1f}%), expected "{}"'

for pred_dict, expec in zip(predictions, expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print(template.format(glovedata.GESTURES[class_id],
                          100 * probability, expec))

In [ ]:
export_dir = classifier.export_savedmodel(
    export_dir_base="model_export",
    serving_input_receiver_fn=serving_input_receiver_fn)
print('Exported to:', export_dir)